In [15]:
from qiskit.aqua.algorithms import VQE, ExactEigensolver
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit import IBMQ
from qiskit.providers.aer import noise
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter

In [43]:
def get_qubit_op(dist):
    driver = PySCFDriver(atom="H 0.0 0.0 0.0; H 0.0 0.0 " + str(dist), unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2

    # Build the qubit operator, which is the input to the VQE algorithm in Aqua
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    map_type = 'PARITY'
    qubitOp = ferOp.mapping(map_type)
    qubitOp = qubitOp.two_qubit_reduced_operator(num_particles)
    num_qubits = qubitOp.num_qubits
    return qubitOp, num_particles, num_spin_orbitals, shift

In [58]:
import timeit
start_time = timeit.default_timer()

backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.2, 2.90, 0.05)
exact_energies = []
vqe_energies = []
optimizer = COBYLA(maxiter=30)
for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
    result = ExactEigensolver(qubitOp).run()
    exact_energies.append(result['energy'] + shift)
    initial_state = HartreeFock(
        qubitOp.num_qubits,
        num_spin_orbitals,
        num_particles,
        'parity'
    ) 
    var_form = UCCSD(
        qubitOp.num_qubits,
        depth=1,
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='parity'
    )
    vqe = VQE(qubitOp, var_form, optimizer, 'matrix')
    results = vqe.run(backend)['energy'] + shift
    vqe_energies.append(results)
    print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", results, "Exact Energy:", exact_energies[-1])
    
print("All energies have been calculated")
#calculate the time
elapsed = timeit.default_timer() - start_time

/anaconda3/envs/ibm/lib/python3.7/site-packages/qiskit/aqua/operators/base_operator.py:182: DeprecationWarning: two_qubit_reduced_operator method is deprecated and it will be removed after 0.6. Now it is moved to the `Z2Symmetries` class as a classmethod. Z2Symmeteries.two_qubit_reduction(num_particles)
  DeprecationWarning)
/anaconda3/envs/ibm/lib/python3.7/site-packages/qiskit/aqua/algorithms/adaptive/vqe/vqe.py:105: DeprecationWarning: operator_mode option is deprecated and it will be removed after 0.6. Now the operator has its own mode, no need extra info to tell the VQE.
  "Now the operator has its own mode, no need extra info to tell the VQE.", DeprecationWarning)


Interatomic Distance: 0.2 VQE Result: -2.3242234017660315 Exact Energy: -2.3242289077399385


/anaconda3/envs/ibm/lib/python3.7/site-packages/qiskit/aqua/operators/base_operator.py:182: DeprecationWarning: two_qubit_reduced_operator method is deprecated and it will be removed after 0.6. Now it is moved to the `Z2Symmetries` class as a classmethod. Z2Symmeteries.two_qubit_reduction(num_particles)
  DeprecationWarning)
/anaconda3/envs/ibm/lib/python3.7/site-packages/qiskit/aqua/algorithms/adaptive/vqe/vqe.py:105: DeprecationWarning: operator_mode option is deprecated and it will be removed after 0.6. Now the operator has its own mode, no need extra info to tell the VQE.
  "Now the operator has its own mode, no need extra info to tell the VQE.", DeprecationWarning)


Interatomic Distance: 0.25 VQE Result: -2.2648005266481936 Exact Energy: -2.264803734319095


/anaconda3/envs/ibm/lib/python3.7/site-packages/qiskit/aqua/operators/base_operator.py:182: DeprecationWarning: two_qubit_reduced_operator method is deprecated and it will be removed after 0.6. Now it is moved to the `Z2Symmetries` class as a classmethod. Z2Symmeteries.two_qubit_reduction(num_particles)
  DeprecationWarning)
/anaconda3/envs/ibm/lib/python3.7/site-packages/qiskit/aqua/algorithms/adaptive/vqe/vqe.py:105: DeprecationWarning: operator_mode option is deprecated and it will be removed after 0.6. Now the operator has its own mode, no need extra info to tell the VQE.
  "Now the operator has its own mode, no need extra info to tell the VQE.", DeprecationWarning)


...

Interatomic Distance: 2.75 VQE Result: -0.9625049944389286 Exact Energy: -0.962602052383793


/anaconda3/envs/ibm/lib/python3.7/site-packages/qiskit/aqua/operators/base_operator.py:182: DeprecationWarning: two_qubit_reduced_operator method is deprecated and it will be removed after 0.6. Now it is moved to the `Z2Symmetries` class as a classmethod. Z2Symmeteries.two_qubit_reduction(num_particles)
  DeprecationWarning)
/anaconda3/envs/ibm/lib/python3.7/site-packages/qiskit/aqua/algorithms/adaptive/vqe/vqe.py:105: DeprecationWarning: operator_mode option is deprecated and it will be removed after 0.6. Now the operator has its own mode, no need extra info to tell the VQE.
  "Now the operator has its own mode, no need extra info to tell the VQE.", DeprecationWarning)


Interatomic Distance: 2.8 VQE Result: -0.9588910358936278 Exact Energy: -0.9589679446520953


/anaconda3/envs/ibm/lib/python3.7/site-packages/qiskit/aqua/operators/base_operator.py:182: DeprecationWarning: two_qubit_reduced_operator method is deprecated and it will be removed after 0.6. Now it is moved to the `Z2Symmetries` class as a classmethod. Z2Symmeteries.two_qubit_reduction(num_particles)
  DeprecationWarning)
/anaconda3/envs/ibm/lib/python3.7/site-packages/qiskit/aqua/algorithms/adaptive/vqe/vqe.py:105: DeprecationWarning: operator_mode option is deprecated and it will be removed after 0.6. Now the operator has its own mode, no need extra info to tell the VQE.
  "Now the operator has its own mode, no need extra info to tell the VQE.", DeprecationWarning)


Interatomic Distance: 2.85 VQE Result: -0.9553916021513149 Exact Energy: -0.9554861867363298
All energies have been calculated


In [59]:
elapsed

48.08540111599996